# neu 350 spring 2021<br>week 5: analyzing the Drosophila courtship song

## part 2: systematically plotting the fly song annotations contained in a single directory





## 1. Loading Libraries

In [1]:
# Import packages

import os
import numpy as np
import matplotlib.pyplot as pl
import scipy as sp
from scipy.io import loadmat
import glob    ####  this will help us load all the files with specific extension in a given folder

In [2]:
import matplotlib
%matplotlib notebook

# 4. If you want to plot data from all the files using a 'for' loop for quick visualization 

In [5]:
## below, supply the path to the folder containing your .mat files.

## '.' means current directory, so this will work if the `dataset 1` folder is inside the folder 
## that contains this notebook

## suggestion: separate individual experiments into their own folders before running this step.
## each experiment is identified by what comes before the _ in the .mat files, and corresponds to a single .abf file

list_of_all_files=(glob.glob("./dataset 1/*.mat")) ## this will make a list consisting of all the files with .mat extension

## let's print the number of elements in your list (i.e. the number of .mat files)
print(f"Collected filenames for {len(list_of_all_files)} files.")

## for example, let's print the name of the second element of this list
print('The second element of the list is', list_of_all_files[1])

##
print(os.path.basename(os.path.normpath(list_of_all_files[1])))

Collected filenames for 21 files.
The second element of the list is ./dataset 1/18n19004_6.mat
18n19004_6.mat


In [ ]:
for file in list_of_all_files:
    mat=loadmat(file)

    raw, times, sines, pulses = mat['dataout'], mat['times'], mat['sines'], mat['pulses']
    times = times.squeeze()
    
    # make a plot of the fly song, with pulse and sine song annotated by color
    # make a second plot of the carrier frequencies of detected sine and pulse song.

    fig,axs = pl.subplots(3, 1, sharex=True)

    # plot raw mic trace
    axs[0].plot(times,raw[:,0], color='grey') 
    axs[0].set_ylabel('Audio amplitude (a.u.)')

    #unpack sine annotation
    sine_starts = sines[0].astype(int)     # start coord of a sine song bout
    sine_ends = sines[1].astype(int)       # end coord of a sine song bout

    sine_cfs = sines[2]                    # carrier freq of that bout
    sine_cft = times[((sine_starts + sine_ends)/2).astype(int)] # plot CF at the center of the detected feature

    #unpack pulse annotation
    pulse_starts = pulses[1].astype(int)   # start coord of an individual pulse
    pulse_ends = pulses[2].astype(int)     # end coord of an individual pulse

    pulse_cfs = pulses[3]                  # carrier freq of that individual pulse
    pulse_cft = times[((pulse_starts + pulse_ends)/2).astype(int)] # time coord at center of that pulse


    #paint sines red, pulses blue
    for i in range(len(sine_starts)):
        sine_line = axs[0].plot(times[sine_starts[i]:sine_ends[i]],
                                raw[sine_starts[i]:sine_ends[i],0],
                                color='red')

    for i in range(len(pulse_starts)):
        pulse_line =axs[0].plot(times[pulse_starts[i]:pulse_ends[i]],
                                raw[pulse_starts[i]:pulse_ends[i],0],
                                color='dodgerblue')

    #legend
    axs[0].scatter([], [], c="red", label="sine song", s=20)  # make an empty scatter plot to generate the legend from
    axs[0].scatter([], [], c="dodgerblue", label="pulse song", s=20)
    axs[0].legend(loc="best")
    axs[0].set_title(file)
    axs[0].set_title(os.path.basename(os.path.normpath(file)))

    # second y axis (dots): carrier frequencies
    axs[1].plot(sine_cft, sine_cfs, marker='.', color='red', linewidth=0)
    axs[1].plot(pulse_cft, pulse_cfs, marker='.', color='dodgerblue', linewidth=0)
    axs[1].set_xlabel('Time (sec)')
    axs[1].set_ylabel('Carrier frequency (Hz)')
    
    # third y axis: LED stimulus
    axs[2].plot(times,raw[:,1], color='black')
    axs[2].set_ylabel('LED stmulation (V)')
    axs[2].set_ylim((-0.5,10.5))  # this trace has no stim!
    axs[2].set_xlabel('Time (sec)')